In [2]:
import pandas as pd
import numpy as np
import pprint
import os
from datetime import datetime
import glob
import pandas as pd
import emgimporter

In [3]:
path=['emg_data','ipa_emg']
glob.glob(os.path.join(os.path.expanduser('~'),*path,'**'),recursive=True)
fList = [(os.path.basename(f.split('-',1)[0]), datetime.strptime(f.split('-',1)[1],"%Y%m%d-%H%M%S"), f)
          for f in glob.glob(os.path.join(os.path.expanduser('~'),*path,'**'),recursive=True)
          if not f.startswith('.') and not os.path.isdir(f)]
print(fList[2])

('markus_7', datetime.datetime(2017, 7, 8, 14, 4, 14), '/home/myo/emg_data/ipa_emg/markus_20170708/markus_7-20170708-140414')


In [4]:
dic = { 1: 'Tasse aufnehmen', 
        2: 'Tasse halten',
        3: 'Tasse abstellen',
        4: 'Tasse hoch&runter',
        8: 'Ruhe (Supination)',
        9: 'Ruhe (Pronation)'
      }
its = ['cupv1','cupv2','kettlev1','data','loosev1']

In [5]:
feed_dic = emgimporter.import_folder(its,dic,path=['emg_data','ipa_emg'])

Create LSTM compatible file sequences
----------------------------------------------
Recognize Gestures 1,2,3 sth

In [6]:
# Calculate basic number of samples per sequence (== look back time)
look_back_time = 3 # [s]
EMG_freq = 200 # [Hz]

nb_samples = look_back_time * EMG_freq
print(nb_samples)

600


In [7]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LocallyConnected2D
from keras.layers import Conv2D, MaxPooling2D, LSTM, Conv1D, MaxPooling1D, ZeroPadding2D, GRU
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [16]:
# create test
data = [[i for i in range(100)]]
data = np.array(data,dtype=float)
target = [[i for i in range(1,101)]]
target = np.array(target,dtype=float)

data = data.reshape((1,1,100))
target = target.reshape((1,1,100))

x_test = [i for i in range(100,200)]
x_test = np.array(x_test).reshape((1,1,100))
y_test = [i for i in range(101,201)]
y_test = np.array(y_test).reshape(1,1,100)

In [17]:
look_back = 1

In [18]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(1,100),return_sequences=True))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_absolute_error',optimizer= 'adam',metrics=['accuracy'])

In [19]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 100)            80400     
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1)              101       
Total params: 80,501.0
Trainable params: 80,501
Non-trainable params: 0.0
_________________________________________________________________


In [21]:
lstm_model.fit(data,target,epoch=100,batch_size=1, verboose=2,validation_data=(x_test,y_test))

TypeError: Unrecognized keyword arguments: {'epoch': 100, 'verboose': 2}